In [5]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import bocd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import yfinance as yf

from machine_learning_finance import (analyze_trades, 
    calc_probabilties_without_lookahead, plot_backtest_analysis, 
    metrics_to_dataframe, create_train_test_windows, make_inverse_env_for,
    calculate_polynomial_regression, TraderEnv)

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]

symbol = "QQQ"
file = f"../backtests/{symbol}-model-back-test.csv"
period = 365
pd.options.display.max_rows = None
inverse = None # set to anything to graph inverse longs

def plot_win_loss(file, metrics):
    ledger = pd.read_csv(file)
    # Set the style for the plots
    sns.set(style='whitegrid')

    # Create a bar plot for profit and loss stats
    profit_stats, loss_stats = metrics['profit_stats'], metrics['loss_stats']
    profit_loss_df = pd.DataFrame([profit_stats, loss_stats], columns=['min', 'max', 'mean', 'median', 'std'], index=['profit', 'loss'])

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=profit_loss_df.transpose(), palette='muted')
    ax.set_title('Profit and Loss Statistics')
    ax.set_ylabel('Value')

    # Show the plot
    plt.show()


        
def analyze_and_graph_range(file, symbol, start, end, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, start=start, end=end)    
    test_df = calc_probabilties_without_lookahead(test_df, hist_df, hist_cutoff="2023-05-15")
    plot_backtest_analysis(test_df, ledger, inverse=symbol)
    return ledger, test_df
   
def analyze_and_graph(file, symbol, period, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, None, 365 * 4, None, 365)
    env = TraderEnv(symbol, test_df, hist_df)
    
    # Todo here: we have all the data we need to render our graphs in the env already
    # We need to modify plot_backtest_analysis to:
    # * render change points
    # * render new moving averages
    # * remove all the probability stuff
    # * Figure out how to supply the polynomials!!!
    
    fig = plot_backtest_analysis(env.orig_timeseries_with_analytics, ledger, inverse=inverse)

    
    fig.update_layout(title='Backtest Analysis', xaxis_title='Date', height=800)
    
    
    fig.show()

    return ledger, test_df

ticker_obj = yf.download(tickers=symbol)
df = pd.DataFrame(ticker_obj)


ledger, df2 = analyze_and_graph(file, symbol, 365*4, df)    
#ledger, df2 = analyze_and_graph_range(file, symbol, "03-27-2020", "12-17-2021", df)

# Display metrics as text
metrics = analyze_trades(ledger, symbol, period)
metrics["file"] = file
metrics_df = metrics_to_dataframe(metrics) 
metrics_df
#df2[["moving_avg", "change_points", "rt", "Close"]]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


,test,duration_min,duration_max,duration_mean,duration_median,duration_std,total_return,buy_and_hold_performance,volatility,maximum_drawdown,...,profit_std,loss_min,loss_max,loss_mean,loss_median,loss_std,patience_min,patience_max,patience_mean,patience_median
0,../backtests/QQQ-model-back-test.csv,1,281,41.916667,11.0,78.808696,42.017003,9.07934,0.893602,0.105414,...,4.653745,-6.38317,-0.071407,-3.154137,-3.080986,3.012526,-15.402005,17.491131,-1.772741,-1.461655


In [6]:
ledger

,Unnamed: 0,Date,Product,Side,Action,Profit_Percent,Profit_Actual,Fee,Value,Price,Shares,Cost,Daily_Return,Cum_Return,Rolling_Max,Drawdown
0,0,2022-03-02,QQQ,long,enter,0.000000,0.000000,12.500000,4938.889200,350.692201,14.0,4909.690817,NaN,NaN,NaN,NaN
1,0,2022-03-03,QQQ,long,exit,-1.428487,-82.233171,12.098891,4905.266829,345.682610,0.0,0.000000,-0.006808,0.993192,0.993192,0.000000
2,0,2022-03-03,QQQ,short,enter,0.000000,0.000000,12.263167,4940.920064,345.682610,0.0,0.000000,0.007268,1.000411,1.000411,0.000000
3,0,2022-05-10,QQQ,short,exit,12.125285,576.178134,10.631866,5469.181796,303.767610,0.0,4263.378404,0.106916,1.107371,1.107371,0.000000
4,0,2022-05-10,QQQ,long,enter,0.000000,0.000000,13.672954,5404.379640,303.767610,17.0,5164.049368,-0.011849,1.094250,1.107371,0.013121
5,0,2022-06-21,QQQ,long,exit,-6.543431,-349.971357,12.065358,5105.537485,283.890786,0.0,0.000000,-0.055296,1.033742,1.107371,0.073629
6,0,2022-06-21,QQQ,short,enter,0.000000,0.000000,12.763844,5140.557239,283.890786,0.0,0.000000,0.006859,1.040833,1.107371,0.066538
7,0,2022-06-22,QQQ,short,exit,0.145856,-5.008521,12.047760,5087.765120,283.476714,0.0,4831.151891,-0.010270,1.030144,1.107371,0.077227
8,0,2022-06-22,QQQ,long,enter,0.000000,0.000000,12.719413,5027.331805,283.476714,17.0,4819.104131,-0.011878,1.017907,1.107371,0.089463
9,0,2022-08-19,QQQ,long,exit,15.031877,710.543053,13.858765,5785.588760,326.088585,0.0,0.000000,0.150827,1.171435,1.171435,0.000000
